In [1]:
import copy
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from bcmix import *

c:\Users\Andrew\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fed = pd.read_csv("realdata/FEDFUNDS.csv")
pce = pd.read_csv("realdata/EXPINF1YR.csv")

In [3]:
X = fed["FEDFUNDS"].values[48:]
y = pce["EXPINF1YR"].values[48:]

In [4]:
CP = 91

XX = X[:CP].reshape((-1, 1))
yy = y[:CP]
model0 = LinearRegression().fit(XX, yy)
yy_hat = model0.predict(XX)
res0 = (yy - yy_hat).reshape((-1, 1))

XX = X[CP : len(X)].reshape((-1, 1))
yy = y[CP : len(X)]
model1 = LinearRegression().fit(XX, yy)
yy_hat = model1.predict(XX)
res1 = (yy - yy_hat).reshape((-1, 1))

In [5]:
fed.iloc[91 + 48]

observation_date    2021-08-01
FEDFUNDS                  0.09
Name: 139, dtype: object

In [6]:
SIGMA = ((res0.T @ res0 + res1.T @ res1) / len(y)).item() ** 0.5
ALPHA0 = model0.intercept_ / SIGMA
ALPHA1 = model1.intercept_ / SIGMA
BETA0 = model0.coef_.item() / SIGMA
BETA1 = model1.coef_.item() / SIGMA

In [7]:
err = np.concatenate([res0.flatten(), res1.flatten()]) / SIGMA

In [8]:
ACTION_RANGE = (0.0, 10.0)
N_TRIALS = 25
DATA_LEN = len(y)
P = 0.025

In [ ]:
mean_true = np.array([[2.0], [0.0]])
covm_true = np.array([[1.0, 0.0], [0.0, 1.0]])
# prior
canonical_0 = np.array([[0.0], [0.0]])
precision_0 = np.array([[1.0, 0.0], [0.0, 1.0]])
for i in range(48):
    x = fed["FEDFUNDS"].iloc[i]
    X = np.array([[1, x]])
    y = pce["EXPINF1YR"].iloc[i] / SIGMA
    canonical_0 = canonical_0 + X.T * y
    precision_0 = precision_0 + X.T @ X

logcon_0 = (np.linalg.slogdet(precision_0)[1] - (canonical_0.T @ np.linalg.inv(precision_0) @ canonical_0).item()) / 2
states = {0: {"can": canonical_0, "pre": precision_0, "log": logcon_0, "pit": 0.0}}

In [ ]:
def objective_bcmix(trial):
    a = trial.suggest_float('a', ACTION_RANGE[0], ACTION_RANGE[1])
    q = q_myopic_with_change(states_i, a, alpha_i, beta_i, mean_true, covm_true, p=P)
    return q

In [ ]:
for i in range(1):
    simresult_i = []
    canonical_i, precision_i, states_i = canonical_0, precision_0, copy.deepcopy(states)

    for j in range(DATA_LEN):
        alpha_i = ALPHA0 if j < CP else ALPHA1
        beta_i = BETA0 if j < CP else BETA1
        study_bcmix = optuna.create_study(direction="maximize")
        study_bcmix.optimize(objective_bcmix, n_trials=N_TRIALS)
        a_bcmix = study_bcmix.best_trial.params['a']
        y_bcmix, alpha_i, beta_i = env_response(a_bcmix, alpha_i, beta_i, mean_true=None, covm_true=None, p=0, err=err[j])
        states_i = update_with_change(states_i, a_bcmix, y_bcmix, p=P)
        simresult_i.append([a_bcmix, y_bcmix * SIGMA])